In [1]:
import sys
import os
from scipy import linalg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.graph_objects as go
sys.path.append("..")
# Enable interactive plotting in Jupyter Notebook

from utils.camera_utils import *

In [2]:
run_name = "RUN2"

spiegel_calib = "spiegel.mp4"
front_calib = "front.mp4"
stereo_calib = "stereo.mp4"

In [3]:
calibration_dir = f"../videos/{run_name}/calibration"
detection_dir = f"../videos/{run_name}/detection"
detection_files = os.listdir(detection_dir)
sC = stereoCamera(camera_size={0: (480, 240), 1: (480, 240)})
vL = videoLoader()


# Setting the Camera Positions

In [4]:
vL.load_video(os.path.join(detection_dir, detection_files[0]))
frame = vL[10]
# 0 = mirror-cam 1 = frontal-cam
sC.set_anchor_point(frame, 0)
sC.set_anchor_point(frame, 1)
for frame in vL[:100]:
    frame = sC.draw_camera_region(frame)
    cv2.imshow("Frame", frame)
    frame0, frame1 = sC(frame)
    cv2.waitKey(10)
cv2.destroyAllWindows()


Left mouse button pressed! (632, 95)
Left mouse button pressed! (640, 103)
Left mouse button pressed! (634, 99)
Left mouse button pressed! (660, 456)


# Mirror Calibration

In [8]:
vL.load_video(os.path.join(calibration_dir, spiegel_calib))
frames = vL[100:110]
sC.calibrate(frames,0, rows=5, columns=6)

Left mouse button pressed! [(758, 221)]
Left mouse button pressed! [(758, 221), (1126, 473)]
ret False
ret False
ret False
ret True
ret True
ret True
ret False
ret False
ret False
ret False
rmse: 0.1642170622286934
camera matrix:
 [[385.63192176   0.         162.07463747]
 [  0.         399.76064182 137.283311  ]
 [  0.           0.           1.        ]]
optimized camera matrix:
 [[351.17431698   0.         180.38545863]
 [  0.         208.20744321 101.8867699 ]
 [  0.           0.           1.        ]]
distortion coeffs:
 [[-1.10375559e+00  1.33907035e+01  2.14867927e-02  7.85564484e-02
  -8.46277323e+01]]


# Frontal Calibration

In [11]:
vL.load_video(os.path.join(calibration_dir, front_calib))
sC.calibrate(frames,1, rows=5, columns=6)

Left mouse button pressed! [(1344, 185)]
Left mouse button pressed! [(1344, 185), (935, 529)]
ret False
ret False
ret False
ret False
ret False
ret False
ret False
ret False
ret False
ret False
No Corners were detected, failed calibration
